# Improving Agentic Systems with RAG and Prompt Engineering Exercise

In this notebook, we'll explore how to use prompt engineering and RAG to improve agentic system performance. First use the open-source model Zephyr-7b beta (a fine tuned version of Mistral-7B, developed by mistral.ai) to observe how changes in prompts influence the model's output. Then, we'll build a RAG system from scratch to help our system provide correct responses.

Below, we filter out any annoying warning messages that might pop up when using certain libraries. Your code will run the same without doing this, we just prefer a cleaner output.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Part 1: Prompt Engineering - Designing a Prompt for Text Classification

We'll follow the below workflow we saw in module 2 to install apppropriate packages and instantiate our model.

## 🔧 Step 1: Install Required Packages
Install `transformers` and `accelerate`.

In [ ]:
%%capture
!pip install -U transformers 
!pip install -U accelerate

## 🤖 Step 2: Import Required Libraries
Import from `transformers`:
* `AutoModelForCausalLM`
* `AutoTokenizer`
* `pipeline`

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

## 🔄 Step 3: Load the Tokenizer
Use `AutoTokenizer` to load the tokenozer for `HuggingFaceH4/zephyr-7b-beta`

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta")

## 🧠 Step 4: Load the model - Zephyr-7b-Beta (a version of Mistral 7B)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "HuggingFaceH4/zephyr-7b-beta",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)


## 🗞 Step 5: Wrap everything in a pipeline object

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=500,
    do_sample=False,

)

## Engineering a Prompt for Text Classification

## 🌟 Activity
Create a prompt that guides a text generation model to classify a piece of text into categories related to food. 

For example, classify text into categories like 'Cuisine Type,' 'Cooking Techniques,' 'Ingredients,' or 'Food Culture.' Define the persona, task, and expected output format in your prompt. Use the following sample text to test your prompt:

Sample Text: "Spaghetti carbonara is a classic Italian pasta dish made with eggs, cheese, pancetta, and pepper. It originated in the Lazio region of Italy and is known for its creamy texture, achieved without using cream. The dish is quick to prepare and relies on high-quality ingredients for the best flavor."

## Chain of Thought Prompting

## 🌟 Activity

Get the model to try and solve the following questions, first without COT, and then using COT:

1. Lily has twice as many apples as John. Together, they have 36 apples. How many apples does each of them have?
2. There are five houses in a row, each painted a different color. In each house lives a person with a different profession. The teacher lives in the red house. The doctor lives next to the teacher. The engineer lives two houses away from the teacher. Who lives in the blue house?

Ensure the model generates the correct output.


# Part 2: Implementing a RAG System

&#x1F6A7; **You will need to restart the notebook kernel before starting this part of the demo otherwise you will run out of GPU memory**

In [ ]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

Below, we filter out any annoying warning messages that might pop up when using certain libraries. Your code will run the same without doing this, we just prefer a cleaner output.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

We'll follow the below workflow to install apppropriate packages and instantiate our model.

## 🔧 Step 1: Install Required Packages


In [ ]:
%%capture
!pip install langchain 
!pip install langgraph
!pip install langchain-huggingface 
!pip install -U langchain-community
!pip install faiss-gpu 
!pip install transformers 
!pip install accelerate 
!pip install datasets 
!pip install sentence-transformers

## 🤖 Step 2: Import Required Libraries
Complete the below imports

In [ ]:
from transformers 
from langchain_huggingface 
from langchain.text_splitter 
from langchain.schema
from langchain.vectorstores 
from langchain_huggingface
from datasets 
from langgraph.graph 
from langchain.schema.runnable
from typing 

## 🔄 Step 3: Load the Tokenizer

Load the tokenizer for `microsoft/phi-3-mini-4k-instruct`

## 🧠 Step 4: Load the model - Phi 3 Mini 4k Instruct

Load the model for `microsoft/phi-3-mini-4k-instruct`

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

## 🗞 Step 5: Wrap everything in a pipeline object

## &#x1F4F0; Step 6: Process the Documents that will form the Knowledge Base

Use the `sentence-transformers/natural-questions` dataset from Huggingface (originally Google, info here https://ai.google.com/research/NaturalQuestions) to create a list of documents called `docs`, then use the `CharacterTextSplitter` to split the documents into chunks of size 500 with a 50 character overlap.

## &#x1F522; Step 7: Embed the Documents in the Vector Database

Use the `all-MiniLM-L6-v2` model to embed the chunks of text and create a retreiver to query the vectorstore, setting the number of documents to retrieve as 3.

## &#x1F531; Step 8: Define Graph Components

Define a `RAGState` class so it expects a question, possibly some helpful documents and eventually an answer.

Then define two functions representing our nodes: 
* `retrieve` , which gets the current question from the state, searches for documents similar to the question in the vectorstore, and then updates the state so that docs points to the relevant documents we found.
* `generate` , which gets the current question and docs from the state, constructs a prompt so that it contains the relevant documents that were found, the question, and a slot for the answer, then pass the prompt to our language model. Update the state so that answer contains the response from the model.

## &#x25B6; Step 9: Build Graph

Construct a graph where the flow of events are retrieve -> generate -> answer.

## 🧪 Step 10: Test the System by Asking a Question it Should Know About
Try the system with the question "What do the red stripes mean on the american flag?"

## ✅❌ Step 11: Evaluate System Performance
Take the evaluation code from the demo and see if it works for the new dataset

## Stretch activities 🌟

Now that you’ve built a system where an LLMs responses are grounded by a RAG system, here are some suggested activities to build your knowledge:

1. Swap the open-source HuggingFace LLM for a GPT 3.5 agent. You'll need to adapt the model loading code and the functions that define the nodes of the graph.
2. Build the RAG system into a multi agent swarm.
3. Figure out a better way of evaluating the response of LLM with respect to the ground truth.